<a href="https://colab.research.google.com/github/steinjm21/ColabInClass/blob/main/matplotlibProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone https://github.com/steinjm21/NumpyMatplotlib.git

fatal: destination path 'NumpyMatplotlib' already exists and is not an empty directory.


In [26]:
import numpy as np
import sys
import datetime
from mpi4py import MPI

ModuleNotFoundError: No module named 'mpi4py'

In [ ]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count

In [ ]:
def main():
    comm = MPI.COMM_WORLD
    cpus = comm.Get_size()
    rank = comm.Get_rank()
    n_samples = int(100000)
    if rank == 0:
      start_time = datetime.datetime.now()
      partitions = [ int(n_samples / cpus) ] * cpus
      counts = [ int(0) ] * cpus
    else:
      partitions = None
      counts = None
    partition_item = comm.scatter(partitions, root=0)
    count_item = inside_circle(partition_item)
    counts = comm.gather(count_item, root=0)
    my_pi = 4.0 * counts / n_samples
    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    size_of_float = np.dtype(np.float64).itemsize
    memory_required = 3 * n_samples * size_of_float / (1024**3)
    print("Pi: {}, memory: {} GiB, time: {} s".format(my_pi, memory_required, elapsed_time))

In [ ]:
if __name__ == '__main__':
    main()